# Multivariate Linear Regression

* [강의출처](https://www.youtube.com/watch?v=1JT8KhvymmY&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=6)
* [코드출처](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-04_1_multivariable_linear_regression.ipynb)

## Module

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Naive Data Representation

### Data Load

In [2]:
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

### Weights

In [3]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)
nb_epochs = 1000

In [4]:
%%time
for epoch in range(1, nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()
        ))

Epoch  100/1000 w1: 0.673 w2: 0.676 w3: 0.676 b: 0.008 Cost: 1.564299
Epoch  200/1000 w1: 0.679 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.498252
Epoch  300/1000 w1: 0.684 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.435646
Epoch  400/1000 w1: 0.689 w2: 0.678 w3: 0.678 b: 0.008 Cost: 1.376286
Epoch  500/1000 w1: 0.694 w2: 0.678 w3: 0.678 b: 0.009 Cost: 1.320047
Epoch  600/1000 w1: 0.699 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.266736
Epoch  700/1000 w1: 0.704 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.216188
Epoch  800/1000 w1: 0.709 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.168284
Epoch  900/1000 w1: 0.713 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.122858
Epoch 1000/1000 w1: 0.718 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.079801
Wall time: 336 ms


## Matrix Representation

In [5]:
x_train = torch.stack([x1_train, x2_train, x3_train], dim=2).squeeze()
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [6]:
x_train.shape, y_train.shape

(torch.Size([5, 3]), torch.Size([5, 1]))

In [7]:
W = torch.zeros((x_train.shape[-1], 1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)
optimizer = optim.SGD([W, b], lr=1e-5)
nb_epochs = 20

In [8]:
%%time
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712646
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371017
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

In [9]:
hypothesis.squeeze().detach()

tensor([152.8020, 183.6731, 180.9677, 197.0699, 140.1000])

## Using Pytorch Method

In [10]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # input shape, output shape

    def forward(self, x):
        return self.linear(x)

model = MultivariateLinearRegressionModel()

In [11]:
# init
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 20

In [12]:
for epoch in range(1, nb_epochs+1):
    h = model(x_train)
    c = F.mse_loss(h, y_train)
    
    optimizer.zero_grad()
    c.backward()
    optimizer.step()
    
    #로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, c.item()
    ))

Epoch    1/20 Cost: 8889.902344
Epoch    2/20 Cost: 2787.931152
Epoch    3/20 Cost: 875.286072
Epoch    4/20 Cost: 275.773132
Epoch    5/20 Cost: 87.856453
Epoch    6/20 Cost: 28.954002
Epoch    7/20 Cost: 10.490484
Epoch    8/20 Cost: 4.702549
Epoch    9/20 Cost: 2.887589
Epoch   10/20 Cost: 2.317985
Epoch   11/20 Cost: 2.138757
Epoch   12/20 Cost: 2.081884
Epoch   13/20 Cost: 2.063361
Epoch   14/20 Cost: 2.056867
Epoch   15/20 Cost: 2.054111
Epoch   16/20 Cost: 2.052568
Epoch   17/20 Cost: 2.051382
Epoch   18/20 Cost: 2.050300
Epoch   19/20 Cost: 2.049285
Epoch   20/20 Cost: 2.048261
